# 上海轨道交通线路可视化与轨迹生成系统 
## 第一部分
- 数据加载与坐标转换（静态操作）
- 该部分代码仅需运行一次，处理耗时的数据读取与转换操作
- 包含坐标转换验证可视化图

In [ ]:
%matplotlib widget

import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyproj import Transformer
import os
import json

# 添加tools_SPCKTrk路径
sys.path.append(r'E:\ResearchDocuments\ROS2WithSPCK\SPCK_Track')  
from tools_SPCKTrk import generate_trajectory

# 设置中文字体
plt.rcParams['font.family'] = 'Microsoft YaHei'   # Windows系统
# plt.rcParams['font.family'] = ['Noto Sans CJK JP']  # Ubuntu系统

# ================== 1. 加载Excel数据 ==================
# 文件路径
file_path = r"E:\ResearchDocuments\ROS2WithSPCK\docs\三制式线路信息汇总.xlsx"
sheet_name = "线路信息汇总"

# 读取Excel文件
df = pd.read_excel(file_path, sheet_name=sheet_name)

# ================== 2. 坐标转换 ==================
# 定义参考点(国家会展中心站)作为坐标原点
reference_point = [121.2882604, 31.1935644]

# 使用pyproj创建坐标转换器
transformer = Transformer.from_crs(
    "EPSG:4326",   # WGS84经纬度
    "+proj=tmerc +lat_0={} +lon_0={} +k=1 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs".format(
        reference_point[1], reference_point[0]
    ),
    always_xy=True
)

# 转换所有点的坐标
local_coordinates = []
for _, row in df.iterrows():
    x, y = transformer.transform(row['longitude'], row['latitude'])
    local_coordinates.append([x, y])

# 转换为numpy数组方便处理
local_coordinates = np.array(local_coordinates)

# 添加局部坐标到DataFrame
df['local_x'] = local_coordinates[:, 0]
df['local_y'] = local_coordinates[:, 1]

# ================== 3. 定义重点站点并转换为局部坐标 ==================
key_stations = {
    '上海虹桥客运站': [121.3133074, 31.1959353],
    '国家会展中心': [121.2882604, 31.1935644],
    '虹桥2号航站楼': [121.3199359, 31.1961374],
    '中春路': [121.3299336, 31.1519780],
    '春申站': [121.3499051, 31.0797510],
    '上海松江站': [121.2258998, 30.9885458]
}

# 转换重点站点坐标
key_stations_local = {}
for name, coords in key_stations.items():
    x, y = transformer.transform(coords[0], coords[1])
    key_stations_local[name] = [x, y]

# ================== 4. 定义站点标注偏移和颜色映射 ==================
# 定义站点标注偏移
station_offsets = {
    '国家会展中心':   (-2000, 0),
    '上海松江站':    (-3000, 0),
    '春申站':        (-2000, 0),
    '上海虹桥客运站': (0, 2000),
    '虹桥2号航站楼':  (2000, 0),
    '中春路':        (2000, 0)
}

# 颜色映射 - 按照要求设置
colors = {'上海17号线': 'green', '市域铁机场联络线': 'blue', '沪苏湖高速铁路': 'red'}

# ================== 5. 坐标转换验证可视化 ==================
plt.figure(figsize=(12, 10))

# 根据线路来源分组绘制
route_groups = df.groupby('线路来源')

# 绘制所有路线
for route_name, group in route_groups:
    color = colors.get(route_name, 'gray')
    # 按照segment_id分组，确保连线正确
    segments = group.groupby('segment_id')
    for _, segment in segments:
        # 按照point_id排序
        segment = segment.sort_values('point_id')
        plt.plot(segment['local_x'], segment['local_y'], '-', color=color, linewidth=1, alpha=0.8, 
                 label=route_name if route_name not in plt.gca().get_legend_handles_labels()[1] else "")
    
    # 绘制该线路的所有点
    plt.scatter(group['local_x'], group['local_y'], s=10, color=color, alpha=0.8)

# 标记重点站点
for name, coords in key_stations_local.items():
    # 绘制站点圆点
    plt.scatter(coords[0], coords[1], s=75, color='gold', edgecolor='black', zorder=10)
    
    # 获取站点偏移量，如果没有设置则使用默认值
    offset_x, offset_y = station_offsets.get(name, (150, 150))
    
    # 使用无箭头引线标注站点名称
    plt.annotate(
        name,
        xy=(coords[0], coords[1]),
        xytext=(coords[0] + offset_x, coords[1] + offset_y),
        arrowprops=dict(arrowstyle='-', color='black', lw=1),
        fontsize=10,
        weight='bold',
        bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'),
        ha='center'
    )

# 设置坐标轴和图例
plt.grid(True, linestyle='--', alpha=0.3)
plt.xlabel('X轴 - 东西方向 (米)')
plt.ylabel('Y轴 - 南北方向 (米)')
plt.title('上海线路网络 - 国家会展中心为原点的局部坐标系 (转换验证 - 上北下南未旋转)')
plt.legend(loc='upper right')

# 设置视图范围 - 调整以显示所有点
plt.xlim(-15000, +15000) 
plt.ylim(-25000, +5000) 

# 添加坐标原点参考标记
plt.axhline(y=0, color='k', linestyle='--', alpha=0.2)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.2)
plt.scatter(0, 0, s=100, color='red', marker='+', zorder=6)

# 显示图形
plt.tight_layout()
plt.show()

# 输出重点站点的局部坐标，便于验证
print("\n重点站点的局部坐标 (上北下南未旋转) :")
for name, coords in key_stations_local.items():
    print(f"{name}: X={coords[0]:.2f}米, Y={coords[1]:.2f}米")


## 第二部分
- 轨道生成与可视化（动态操作）
- 该部分代码可以重复运行，用于测试不同的轨道段定义和旋转角度
- 函数封装版本

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from tools_SPCKTrk import generate_trajectory

# 需要先运行第一部分代码，才能获取这些变量
# df, key_stations_local, colors, station_offsets 在第一部分中定义

# ===================== 辅助函数定义 =====================
def apply_rotation(df, key_stations_local, rot_deg):
    """
    应用坐标旋转变换
    
    Args:
        df: 包含局部坐标的DataFrame
        key_stations_local: 包含重点站点局部坐标的字典
        rot_deg: 旋转角度（度）
        
    Returns:
        df: 添加了旋转坐标的DataFrame
        rotated_stations: 旋转后的站点坐标字典
    """
    # 转换为弧度
    rot_rad = np.radians(rot_deg)
    
    # 应用旋转变换到线路点坐标
    df['rotated_x'] = df['local_x'] * np.cos(rot_rad) - df['local_y'] * np.sin(rot_rad)
    df['rotated_y'] = df['local_x'] * np.sin(rot_rad) + df['local_y'] * np.cos(rot_rad)
    
    # 应用旋转变换到重点站点坐标
    rotated_stations = {}
    for name, coords in key_stations_local.items():
        rx = coords[0] * np.cos(rot_rad) - coords[1] * np.sin(rot_rad)
        ry = coords[0] * np.sin(rot_rad) + coords[1] * np.cos(rot_rad)
        rotated_stations[name] = [rx, ry]
    
    return df, rotated_stations, rot_rad

def generate_track(h_segments, u_segments, L_smo, b_ref, ds, extension_length=15.0):
    """
    生成轨道中心线和钢轨，包括负方向延伸段
    
    Args:
        h_segments: 水平段定义
        u_segments: 超高段定义
        L_smo: 平滑段总长
        b_ref: 轨距
        ds: 积分步长
        extension_length: X轴负方向延伸长度
        
    Returns:
        xvals: 中心线X坐标数组
        yvals: 中心线Y坐标数组
        trajectory_data: 完整轨迹数据字典
    """
    # 调用generate_trajectory生成轨道数据
    trajectory_data, Kappa, U = generate_trajectory(
        h_segments, 
        u_segments, 
        L_smo=L_smo, 
        b_ref=b_ref, 
        ds=ds
    )
    
    # 从trajectory_data中提取数据
    s_vals     = np.array(trajectory_data['s'])
    xvals      = np.array(trajectory_data['x'])
    yvals      = np.array(trajectory_data['y'])
    zvals      = np.array(trajectory_data['z'])
    psi_vals   = np.array(trajectory_data['psi'])
    phi_vals   = np.array(trajectory_data['phi'])
    left_rail  = np.array(trajectory_data['left_rail'])
    right_rail = np.array(trajectory_data['right_rail'])
    
    # 处理X轴负方向延伸
    if extension_length > 0:
        # 创建新的s值序列（从-extension_length到0）
        num_new_points = int(extension_length / ds) + 1  # 包括起点和终点
        s_extension = np.linspace(-extension_length, 0, num_new_points)
        s_extension = s_extension[:-1]  # 移除最后一个点，避免与原始数据第一个点重复
        
        # 创建新的坐标序列
        x_extension = s_extension.copy()  # x = s，沿x轴的直线
        y_extension = np.zeros_like(s_extension)
        z_extension = np.zeros_like(s_extension)
        psi_extension = np.zeros_like(s_extension)
        phi_extension = np.zeros_like(s_extension)
        
        # 计算左右钢轨的坐标
        left_rail_extension = []
        right_rail_extension = []
        
        for i in range(len(s_extension)):
            xC = x_extension[i]
            yC = y_extension[i]
            zC = z_extension[i]
            
            # 对于psi=0的情况，计算钢轨偏移
            half_b = b_ref / 2
            
            # 左股轨道(相对中心线左侧 half_b)
            dxL = 0
            dyL = half_b
            xL = xC + dxL
            yL = yC + dyL
            zL = zC
            
            # 右股轨道(相对中心线右侧 half_b)
            dxR = 0
            dyR = -half_b
            xR = xC + dxR
            yR = yC + dyR
            zR = zC
            
            left_rail_extension.append((xL, yL, zL))
            right_rail_extension.append((xR, yR, zR))
        
        left_rail_extension = np.array(left_rail_extension)
        right_rail_extension = np.array(right_rail_extension)
        
        # 合并新旧数据
        s_vals = np.concatenate([s_extension, s_vals])
        xvals = np.concatenate([x_extension, xvals])
        yvals = np.concatenate([y_extension, yvals])
        zvals = np.concatenate([z_extension, zvals])
        psi_vals = np.concatenate([psi_extension, psi_vals])
        phi_vals = np.concatenate([phi_extension, phi_vals])
        left_rail = np.vstack([left_rail_extension, left_rail])
        right_rail = np.vstack([right_rail_extension, right_rail])
        
        # 对Kappa、U也补零
        kappa_list = []
        u_list = []
        for s_ in s_vals:
            if s_ < 0:
                # 对于新补的(-extension_length,0)段，曲率=0, 超高=0
                kappa_list.append(0.0)
                u_list.append(0.0)
            else:
                # 对于原来的 [0, 轨道结束] 段，按原函数计算
                kappa_list.append(Kappa(s_))
                u_list.append(U(s_))
        
        kappa_vals = np.array(kappa_list)
        u_vals = np.array(u_list)
        
        # 更新trajectory_data
        trajectory_data = {
            's': s_vals.tolist(),
            'x': xvals.tolist(),
            'y': yvals.tolist(),
            'z': zvals.tolist(),
            'psi': psi_vals.tolist(),
            'phi': phi_vals.tolist(),
            'left_rail': left_rail.tolist(),
            'right_rail': right_rail.tolist()
        }
    
    return xvals, yvals, trajectory_data

def plot_railway_map(df, rotated_stations, colors, station_offsets, rot_deg, rot_rad, 
                    track_x, track_y, save_fig=True, fig_name=None,
                    x_min=-500, x_max=2500, y_min=-1000, y_max=4000):
    """
    绘制旋转后的铁路地图，并添加生成的轨道
    
    Args:
        df: 包含旋转坐标的DataFrame
        rotated_stations: 旋转后的站点坐标字典
        colors: 颜色映射字典
        station_offsets: 站点偏移量字典
        rot_deg: 旋转角度（度）
        rot_rad: 旋转角度（弧度）
        track_x: 生成的轨道X坐标数组
        track_y: 生成的轨道Y坐标数组
        save_fig: 是否保存图像
        fig_name: 图像文件名
        x_min: 横坐标显示下限
        x_max: 横坐标显示上限
        y_min: 纵坐标显示下限
        y_max: 纵坐标显示上限
    """
    plt.figure(figsize=(12, 10))
    
    # 分组绘制路线
    route_groups = df.groupby('线路来源')
    for route_name, group in route_groups:
        color = colors.get(route_name, 'gray')
        # 按照segment_id分组，确保连线正确
        segments = group.groupby('segment_id')
        for _, segment in segments:
            # 按照point_id排序
            segment = segment.sort_values('point_id')
            plt.plot(segment['rotated_x'], segment['rotated_y'], '-', color=color, linewidth=1, alpha=0.8, 
                     label=route_name if route_name not in plt.gca().get_legend_handles_labels()[1] else "")
        
        # 绘制该线路的所有点
        plt.scatter(group['rotated_x'], group['rotated_y'], s=10, color=color, alpha=0.8)
    
    # 标记重点站点 - 使用旋转后的坐标
    for name, coords in rotated_stations.items():
        # 绘制站点圆点
        plt.scatter(coords[0], coords[1], s=75, color='gold', edgecolor='black', zorder=10)
        
        # 获取站点偏移量，也需要进行相应旋转以匹配新坐标系
        offset_x, offset_y = station_offsets.get(name, (150, 150))
        rotated_offset_x = offset_x * np.cos(rot_rad) - offset_y * np.sin(rot_rad)
        rotated_offset_y = offset_x * np.sin(rot_rad) + offset_y * np.cos(rot_rad)
        
        # 使用无箭头引线标注站点名称
        plt.annotate(
            name,
            xy=(coords[0], coords[1]),
            xytext=(coords[0] + rotated_offset_x, coords[1] + rotated_offset_y),
            arrowprops=dict(arrowstyle='-', color='black', lw=1),
            fontsize=10,
            weight='bold',
            bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'),
            ha='center'
        )
    
    # 添加生成的轨道中心线
    plt.plot(track_y, track_x, 'k-', linewidth=2, label='Our CenterLine')
    
    # 设置坐标轴和图例
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.xlabel(f'X轴 - 旋转{rot_deg}°后 (米)')
    plt.ylabel(f'Y轴 - 旋转{rot_deg}°后 (米)')
    plt.title(f'上海线路网络 - 国家会展中心为原点，逆时针旋转{rot_deg}°后的坐标系')
    plt.legend(loc='upper right')
    
    # 设置自定义视图范围
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    
    # 添加坐标原点参考标记
    plt.axhline(y=0, color='k', linestyle='--', alpha=0.2)
    plt.axvline(x=0, color='k', linestyle='--', alpha=0.2)
    plt.scatter(0, 0, s=100, color='red', marker='+', zorder=6)
    
    # 显示图形
    plt.tight_layout()
    
    # 保存图形
    if save_fig and fig_name:
        plt.savefig(fig_name, dpi=300, bbox_inches='tight')
        print(f"图像已保存为: {fig_name}")
    
    plt.show()
    
    # 输出重点站点的旋转后坐标
    print(f"\n重点站点旋转{rot_deg}°后的坐标:")
    for name, coords in rotated_stations.items():
        print(f"{name}: X={coords[0]:.2f}米, Y={coords[1]:.2f}米")

def save_rotated_coordinates_csv(df, rot_deg, output_path=None):
    """
    保存旋转后的坐标点到CSV文件
    
    Args:
        df: 包含旋转坐标的DataFrame
        rot_deg: 旋转角度（度）
        output_path: 输出文件路径，如果为None则自动生成
        
    Returns:
        output_path: 保存的文件路径
    """
    # 提取和排序数据
    output_df = df[['自定序号', 'rotated_x', 'rotated_y']]
    output_df = output_df.sort_values('自定序号')
    output_df = output_df.rename(columns={
        'rotated_x': f'X坐标_旋转{rot_deg}度(米)',
        'rotated_y': f'Y坐标_旋转{rot_deg}度(米)'
    })
    
    # 设置默认输出路径
    if output_path is None:
        output_path = f'VirtualRailway_LocalCoordinates_Rotated{rot_deg}deg.csv'
    
    # 保存CSV文件
    output_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"旋转{rot_deg}°后的坐标数据已成功保存至: {output_path}")
    print(f"共导出 {len(output_df)} 个坐标点")
    
    return output_path

def load_self_defined_track_segments_from_excel(file_path, sheet_name="自定义线路"):
    """
    从Excel文件加载轨道段定义并进行验证
    
    Args:
        file_path: Excel文件路径
        sheet_name: 工作表名称，默认为"自定义线路"
        
    Returns:
        h_segments: 水平段定义列表
    """
    import pandas as pd
    import os
    
    # 默认的轨道段定义，以防出错时使用
    default_segments = [
        ('STR', 50),
        ('BLO', 50, 0.0, 300.0),
        ('CIR', 300, 300),
        ('BLO', 50, 300, 0.0),
        ('STR', 50)
    ]
    
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"错误: 文件 '{file_path}' 不存在")
        return default_segments
    
    try:
        # 读取Excel文件
        try:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
        except Exception as e:
            print(f"错误: 无法读取Excel文件 '{file_path}', 工作表 '{sheet_name}': {e}")
            return default_segments
        
        # 检查必要的列是否存在
        required_columns = ["线路分段", "平面线路类型", "Par1", "Par2", "Par3"]
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"错误: Excel文件中缺少必要的列: {', '.join(missing_columns)}")
            return default_segments
        
        # 初始化轨道段列表
        h_segments = []
        
        # 检查分段数是否是4的倍数
        if len(df) % 4 != 0:
            print(f"警告: 线路分段数量 ({len(df)}) 不是4的倍数，每组应有4个分段 (STR, BLO, CIR, BLO)")
            print("尝试按可用的完整组进行处理...")
            # 裁剪到最近的4的倍数
            df = df.iloc[:len(df) - (len(df) % 4)]
            
        # 如果没有有效的分段
        if len(df) == 0:
            print("错误: 没有有效的线路分段")
            return default_segments
        
        # 按组处理分段
        for i in range(0, len(df), 4):
            group = df.iloc[i:i+4].copy()
            
            # 验证分段顺序
            expected_types = ["STR", "BLO", "CIR", "BLO"]
            actual_types = group["平面线路类型"].tolist()
            
            if actual_types != expected_types:
                print(f"警告: 分段组 {(i//4)+1} (分段 {i+1}-{i+4}) 顺序不正确。")
                print(f"  应为: {expected_types}")
                print(f"  实际: {actual_types}")
                print("跳过此组分段...")
                continue
            
            try:
                # 提取STR段
                str_row = group.iloc[0]
                if pd.isna(str_row["Par1"]):
                    print(f"警告: 分段 {str_row['线路分段']} (STR) 缺少Par1参数")
                    continue
                    
                str_params = ("STR", float(str_row["Par1"]))
                
                # 提取第一个BLO段
                blo1_row = group.iloc[1]
                if pd.isna(blo1_row["Par1"]) or pd.isna(blo1_row["Par2"]) or pd.isna(blo1_row["Par3"]):
                    print(f"警告: 分段 {blo1_row['线路分段']} (BLO) 参数不完整")
                    continue
                    
                blo1_params = ("BLO", float(blo1_row["Par1"]), float(blo1_row["Par2"]), float(blo1_row["Par3"]))
                
                # 提取CIR段
                cir_row = group.iloc[2]
                if pd.isna(cir_row["Par1"]) or pd.isna(cir_row["Par2"]):
                    print(f"警告: 分段 {cir_row['线路分段']} (CIR) 参数不完整")
                    continue
                    
                cir_params = ("CIR", float(cir_row["Par1"]), float(cir_row["Par2"]))
                
                # 提取第二个BLO段
                blo2_row = group.iloc[3]
                if pd.isna(blo2_row["Par1"]) or pd.isna(blo2_row["Par2"]) or pd.isna(blo2_row["Par3"]):
                    print(f"警告: 分段 {blo2_row['线路分段']} (BLO) 参数不完整")
                    continue
                    
                blo2_params = ("BLO", float(blo2_row["Par1"]), float(blo2_row["Par2"]), float(blo2_row["Par3"]))
                
                # 进行参数验证
                validation_errors = []
                
                # 验证(2): 检查各段参数数量
                if not pd.isna(str_row["Par2"]) or not pd.isna(str_row["Par3"]):
                    validation_errors.append(f"分段 {str_row['线路分段']} (STR) 应只有1个参数 (Par1)")
                
                if not pd.isna(cir_row["Par3"]):
                    validation_errors.append(f"分段 {cir_row['线路分段']} (CIR) 应只有2个参数 (Par1, Par2)")
                
                # 验证(3): 检查BLO参数的一致性
                
                # 两个BLO的Par1应该相同
                if abs(float(blo1_row["Par1"]) - float(blo2_row["Par1"])) > 1e-6:
                    validation_errors.append(f"两个BLO段 ({blo1_row['线路分段']}和{blo2_row['线路分段']}) 的Par1应相同，"
                                           f"实际为 {blo1_row['Par1']} 和 {blo2_row['Par1']}")
                
                # 第一个BLO的Par3应该与CIR的Par2相同
                if abs(float(blo1_row["Par3"]) - float(cir_row["Par2"])) > 1e-6:
                    validation_errors.append(f"BLO段 {blo1_row['线路分段']} 的Par3应等于CIR段 {cir_row['线路分段']} 的Par2，"
                                           f"实际为 {blo1_row['Par3']} 和 {cir_row['Par2']}")
                
                # 第二个BLO的Par2应该与CIR的Par2相同
                if abs(float(blo2_row["Par2"]) - float(cir_row["Par2"])) > 1e-6:
                    validation_errors.append(f"BLO段 {blo2_row['线路分段']} 的Par2应等于CIR段 {cir_row['线路分段']} 的Par2，"
                                           f"实际为 {blo2_row['Par2']} 和 {cir_row['Par2']}")
                
                # 验证(4): 检查BLO连接直线的参数
                
                # 第一个BLO的Par2应为0
                if abs(float(blo1_row["Par2"])) > 1e-6:
                    validation_errors.append(f"BLO段 {blo1_row['线路分段']} 的Par2应为0，实际为 {blo1_row['Par2']}")
                
                # 第二个BLO的Par3应为0
                if abs(float(blo2_row["Par3"])) > 1e-6:
                    validation_errors.append(f"BLO段 {blo2_row['线路分段']} 的Par3应为0，实际为 {blo2_row['Par3']}")
                
                # 如果有验证错误，打印警告并跳过此组
                if validation_errors:
                    print(f"警告: 分段组 {(i//4)+1} (分段 {i+1}-{i+4}) 存在以下验证错误:")
                    for error in validation_errors:
                        print(f"  - {error}")
                    print("跳过此组分段...")
                    continue
                
                # 验证通过，添加所有段到结果
                h_segments.append(str_params)
                h_segments.append(blo1_params)
                h_segments.append(cir_params)
                h_segments.append(blo2_params)
                
            except (ValueError, TypeError) as e:
                print(f"警告: 处理分段组 {(i//4)+1} (分段 {i+1}-{i+4}) 时出错: {e}")
                print("跳过此组分段...")
                continue
                
        if not h_segments:
            print("错误: 未能从Excel文件中加载任何有效的轨道段定义")
            return default_segments
            
        print(f"成功从Excel文件中加载了 {len(h_segments)} 个轨道段定义")
        return h_segments
    
    except Exception as e:
        print(f"读取轨道段定义时出错: {e}")
        return default_segments
    
# ===================== 线路参数 =====================

# 水平段
# 从Excel文件加载轨道段定义
h_segments = load_self_defined_track_segments_from_excel(
    r"E:\ResearchDocuments\ROS2WithSPCK\docs\跨线运行SPCK曲线定义组.xlsx", 
    sheet_name="自定义线路"
)

# 超高段   # 常值超高 0
u_segments = [ ('CST', 1000,   0.0) ]

# 设置平滑段总长度、轨距、步长等参数
L_smo = 3.0     # 例如3.0 => 左右各1.5m
b_ref = 1.5     # 中心线到钢轨的半轨距
ds    = 5       # 积分步长
extension_length = 15.0  # X 负方向延伸长度

# ===================== 视图设置 与 结果储存 =====================
# 旋转角度（度）- 可修改
RotDeg = 69  # 逆时针旋转角度

"""
# 默认全局视图
X_MIN = -5000
X_MAX = 20000

Y_MIN = -19000
Y_MAX = 6000
"""

# 视图范围设置
X_MIN = -5000
X_MAX = 20000

Y_MIN = -19000
Y_MAX = 6000

# 是否保存CSV文件和图像
SAVE_CSV = False
SAVE_FIG = False

# ===================== 主函数调用 =====================
# 应用坐标旋转
df_rotated, rotated_stations, rot_rad = apply_rotation(df, key_stations_local, RotDeg)

# 生成轨道轨迹
xvals, yvals, trajectory_data = generate_track(h_segments, u_segments, L_smo, b_ref, ds, extension_length)

# 绘制铁路地图
fig_name = f'上海线路网络图_旋转{RotDeg}度.png' if SAVE_FIG else None
plot_railway_map(
    df_rotated, rotated_stations, colors, station_offsets, RotDeg, rot_rad,
    xvals, yvals, SAVE_FIG, fig_name,
    x_min=X_MIN, x_max=X_MAX, y_min=Y_MIN, y_max=Y_MAX
)

# 保存CSV文件
if SAVE_CSV:
    output_path = save_rotated_coordinates_csv(df_rotated, RotDeg)
    print(f"CSV文件已保存至: {output_path}")

In [3]:
# 占位